# Dermatologist-AI Project: Single Network Approach
Workbook for a single network approach that classifies images into melanomas, nevus, or SBK

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path, shuffle):
    data = load_files(path, shuffle=shuffle)
    file_paths = np.array(data['filenames'])
    one_hot_labels = np_utils.to_categorical(np.array(data['target']),3)
    return file_paths, one_hot_labels

#import datasets
train_files, train_labels = load_dataset('../data/train', True)
valid_files, valid_labels = load_dataset('../data/valid', True)
test_files, test_labels = load_dataset('../data/test', False)

# load list of skin condition names
skin_names = [item[14:-1] for item in sorted(glob("../data/train/*/"))]


Using TensorFlow backend.


In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors) 

In [3]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [03:30<00:00, 10.35it/s]


In [5]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

### TODO: Define your architecture.
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', 
                        input_shape=(224, 224, 3)))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv2D(filters=48, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=48, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=48, kernel_size=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 224, 224, 32)      9248      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 224, 224, 32)      1056      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 112, 112, 48)      13872     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 112, 112, 48)      20784     
__________

In [6]:
from keras import optimizers
opt = optimizers.rmsprop(lr=0.00005, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint  

epochs = 50

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(valid_tensors, valid_labels,
            validation_data=(valid_tensors, valid_labels),
            epochs=epochs,
            batch_size=20,
            callbacks=[checkpointer],
            verbose=1)

Train on 150 samples, validate on 150 samples
Epoch 1/50
150/150 [==============================] - 3s - loss: 1.0073 - acc: 0.5200 - val_loss: 1.0318 - val_acc: 0.5200
Epoch 2/50
150/150 [==============================] - 2s - loss: 1.0048 - acc: 0.5200 - val_loss: 1.0327 - val_acc: 0.5200
Epoch 3/50
150/150 [==============================] - 2s - loss: 1.0045 - acc: 0.5200 - val_loss: 1.0325 - val_acc: 0.5200
Epoch 4/50
150/150 [==============================] - 3s - loss: 1.0107 - acc: 0.5200 - val_loss: 1.0267 - val_acc: 0.5200
Epoch 5/50
150/150 [==============================] - 3s - loss: 1.0056 - acc: 0.5200 - val_loss: 1.0157 - val_acc: 0.5200
Epoch 6/50
150/150 [==============================] - 3s - loss: 1.0134 - acc: 0.5200 - val_loss: 1.0217 - val_acc: 0.5200
Epoch 7/50
150/150 [==============================] - 2s - loss: 0.9987 - acc: 0.5200 - val_loss: 1.0368 - val_acc: 0.5200
Epoch 8/50
150/150 [==============================] - 3s - loss: 0.9857 - acc: 0.5200 - val_l

150/150 [==============================] - 3s - loss: 0.9690 - acc: 0.5200 - val_loss: 0.9824 - val_acc: 0.5200
Epoch 18/50
150/150 [==============================] - 3s - loss: 0.9669 - acc: 0.5267 - val_loss: 0.9877 - val_acc: 0.5200
Epoch 19/50
150/150 [==============================] - 3s - loss: 0.9358 - acc: 0.5267 - val_loss: 1.0098 - val_acc: 0.5200
Epoch 20/50
150/150 [==============================] - 3s - loss: 0.9548 - acc: 0.5200 - val_loss: 0.9891 - val_acc: 0.5533
Epoch 21/50
150/150 [==============================] - 3s - loss: 0.9307 - acc: 0.5333 - val_loss: 0.9604 - val_acc: 0.5200
Epoch 22/50
150/150 [==============================] - 2s - loss: 0.9347 - acc: 0.5333 - val_loss: 0.9833 - val_acc: 0.6133
Epoch 23/50
150/150 [==============================] - 3s - loss: 0.9467 - acc: 0.6067 - val_loss: 0.9796 - val_acc: 0.5667
Epoch 24/50
150/150 [==============================] - 3s - loss: 0.9232 - acc: 0.5467 - val_loss: 0.9668 - val_acc: 0.6200
Epoch 25/50
150/150 

150/150 [==============================] - 3s - loss: 0.8642 - acc: 0.6267 - val_loss: 0.8748 - val_acc: 0.6200
Epoch 34/50
150/150 [==============================] - 3s - loss: 0.8406 - acc: 0.6400 - val_loss: 0.9108 - val_acc: 0.6667
Epoch 35/50
 20/150 [===>..........................] - ETA: 2s - loss: 0.6628 - acc: 0.8500

In [ ]:
# Load the model weights with the best validation loss.

model.load_weights('saved_models/best.weights.hdf5')

In [ ]:
import pandas as pd

y_pred = pd.DataFrame(columns=["task_1", "task_2"])

for ii in tqdm(range(len(test_files))):
    path = test_files[ii]
    prediction = np.argmax(network.predict(np.expand_dims(test_tensors[ii], axis=0)))
    if prediction == 0:
        y_pred.loc[path] = [1, 0]
    if prediction == 2:
        y_pred.loc[path] = [0, 1]
    else:
        y_pred.loc[path] = [0, 0]

y_pred.to_csv("predictions.csv")